In [1]:
import shutil
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

# reload notebook automatically after changes to source python files
%load_ext autoreload
%autoreload 2

# change base folder to parent
import os
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(os.getcwd())

# Set CUDA_VISIBLE_DEVICES
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
import pickle

def scale_matches(
    matches: np.ndarray, orig_size: tuple, resize_size: tuple
) -> np.ndarray:
    """Scale match coordinates from resized to original image dimensions."""
    scale = np.array(
        [orig_size[1] / resize_size[1], orig_size[0] / resize_size[0]]  # width scale
    )  # height scale
    return matches * scale[None, :]


def read_mast3r_inliers(path_mast3r_inliers, view1, view2, TOP_K=30):
    with open(path_mast3r_inliers, "rb") as f:
        inliers = pickle.load(f)

    # resize matches_im0 and matches_im1
    matches_im0 = inliers["matches_im0"]
    matches_im1 = inliers["matches_im1"]
    H0, W0 = inliers["H0W0"]
    H1, W1 = inliers["H1W1"]
    
    print(H0, W0, H1, W1)
    

    H0_RESIZE, W0_RESIZE = inliers["H0W0_RESIZE"]
    H1_RESIZE, W1_RESIZE = inliers["H1W1_RESIZE"]
    xyz_0_inliers = inliers["xyz_0"]
    
    print(H0_RESIZE, W0_RESIZE, H1_RESIZE, W1_RESIZE)

    # resize 2d image matches to origina size using the scaling factor H0/H0_RESIZE and W0/W0_RESIZE
    matches_im0 = scale_matches(np.array(matches_im0), (H0, W0), (H0_RESIZE, W0_RESIZE))
    matches_im1 = scale_matches(np.array(matches_im1), (H1, W1), (H1_RESIZE, W1_RESIZE))

    # # resize view1 and view2 to resized size
    # view1 = cv2.resize(view1, (W0_RESIZE, H0_RESIZE))
    # view2 = cv2.resize(view2, (W1_RESIZE, H1_RESIZE))

    # visualize a few matches
    n_viz = TOP_K
    num_matches = matches_im0.shape[0]
    match_idx_to_viz = np.round(np.linspace(0, num_matches - 1, n_viz)).astype(int)
    viz_matches_im0, viz_matches_im1 = (
        matches_im0[match_idx_to_viz],
        matches_im1[match_idx_to_viz],
    )

    viz_imgs = [view1, view2]

    H0, W0, H1, W1 = *viz_imgs[0].shape[:2], *viz_imgs[1].shape[:2]
    img0 = np.pad(
        viz_imgs[0],
        ((0, max(H1 - H0, 0)), (0, 0), (0, 0)),
        "constant",
        constant_values=0,
    )
    img1 = np.pad(
        viz_imgs[1],
        ((0, max(H0 - H1, 0)), (0, 0), (0, 0)),
        "constant",
        constant_values=0,
    )
    stitched_img = np.concatenate((img0, img1), axis=1)

    # Create color array using similar colormap logic
    colors = [
        tuple(int(c * 255) for c in plt.cm.viridis(i / (TOP_K - 1))[:3])
        for i in range(TOP_K)
    ]

    # Draw matches
    for i in range(TOP_K):
        pt0 = tuple(map(int, viz_matches_im0[i]))
        pt1 = (int(viz_matches_im1[i][0] + W0), int(viz_matches_im1[i][1]))

        # Draw line
        cv2.line(stitched_img, pt0, pt1, colors[i], thickness=2)

        # Draw points
        cv2.circle(stitched_img, pt0, 2, colors[i], -1)
        cv2.circle(stitched_img, pt1, 2, colors[i], -1)

    return stitched_img


In [5]:
from src.datasets.dataset_utils import (
    load_tum_poses,
    PoseMode,
)

dataset_root = Path("data/rrc-lab-data/wheelchair-runs-20241220/")
exps_root = Path("results/mast3rvloc-rrclab/")

MAX_ROTATION_ERROR =45.0  # Maximum allowable rotation error in degrees
MAX_TRANSLATION_ERROR = 3.0  # Maximum allowable translation error in meters

experiment_name = f"run-3-query-min-r-{int(MAX_ROTATION_ERROR)}-t-{int(MAX_TRANSLATION_ERROR)}"

exp_folder = exps_root / experiment_name
exp_folder.mkdir(parents=True, exist_ok=True)

ref_scene_folder = dataset_root / "run-1-wheelchair-mapping"
query_scene_folder = dataset_root / "run-3-wheelchair-query"

# Collect and sort RGB and depth files
ref_rgb_files = natsorted(Path(ref_scene_folder / "rgb").glob("*.png"))
# ref_depth_files = natsorted(Path(ref_data_root / "aligned_depth").glob("*.png"))
query_rgb_files = natsorted(Path(query_scene_folder / "rgb").glob("*.png"))
# query_depth_files = natsorted(Path(query_data_root / "aligned_depth").glob("*.png"))

ref_indices, ref_poses = load_tum_poses(
    exp_folder / "retrieved_ref_poses_tum.txt", PoseMode.MAT4x4
)
query_indices, query_gt_poses = load_tum_poses(
    exp_folder / "filtered_query_poses_tum.txt", PoseMode.MAT4x4
)

ref_index = ref_indices[0]
query_index = query_indices[0]

ref_rgb = cv2.imread(str(ref_rgb_files[int(ref_index)]))
query_rgb = cv2.imread(str(query_rgb_files[int(query_index)]))

ref_rgb = cv2.cvtColor(ref_rgb, cv2.COLOR_BGR2RGB)
query_rgb = cv2.cvtColor(query_rgb, cv2.COLOR_BGR2RGB)

mast3r_inliers_file = Path(exp_folder / "mast3r_inliers") / f"query_{int(query_index)}_inliers.pkl"

mast3r_inliers_img = read_mast3r_inliers(mast3r_inliers_file, ref_rgb, query_rgb)

# plot mast3r inliers
plt.figure(figsize=(12, 6))
plt.imshow(mast3r_inliers_img)
plt.axis("off")